In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import common

In [2]:
DHBS = ['Northland', 'Waitemata', 'Auckland', 'Counties Manukau', 'Waikato', 'Bay of Plenty', 'Tairawhiti', 'Lakes','Taranaki', "Hawke's Bay", 'Whanganui', 'MidCentral', 'Hutt Valley', 'Capital and Coast', 'Wairarapa', 'Nelson Marlborough', 'West Coast', 'Canterbury',  'South Canterbury', 'Southern']

In [3]:
datestr='7-april-2020'
today = pd.to_datetime(datestr.replace('-', ' '))
CASE_URL_FORMAT = 'https://www.health.govt.nz/system/files/documents/pages/covid-19-case-details-update-%s.xlsx'


In [4]:
case_url = CASE_URL_FORMAT % datestr
case_url = 'https://www.health.govt.nz/system/files/documents/pages/covid-casedeatails-7april2020.xlsx'

In [5]:
with pd.ExcelFile(case_url) as excel_file:
    cases_dfs = {name: excel_file.parse(i, header=3) for i, name in enumerate(['confirmed', 'probable'])}

In [6]:
def get_case_counts(df, date_name='Report Date', date_col=0):
    return df.groupby([df.columns[date_col], 'DHB']).agg(c=('DHB', 'count')).c.rename_axis([date_name, 'DHB'])

In [7]:
cdf = pd.concat({key: get_case_counts(df) for key, df in cases_dfs.items()}, axis=1, names=['type']).loc[:today - pd.Timedelta(days=1)]

In [8]:
totals_df = cdf.sum(1).rename('cases').reset_index()

In [20]:
px.bar(totals_df, x='Report Date', y='cases', color='DHB', category_orders={'DHB':DHBS})

In [10]:
px.bar(totals_df, color='Report Date', y='cases', x='DHB', color_continuous_scale = px.colors.sequential.algae)

In [11]:
tdf = cdf.unstack('DHB').fillna(0).cumsum().stack('DHB')
tdf['total'] = tdf.sum(1)




In [12]:
ydf = tdf.loc[tdf.index[-1][0], ['confirmed', 'probable']].unstack().reset_index()

In [13]:
px.bar(ydf, y='DHB', x=0, orientation='h', color='type', category_orders={'DHB':DHBS})

In [32]:
xdf = cdf.unstack('DHB').fillna(0).cumsum()
xdf = xdf.stack(['DHB', 'type'])
xdf = xdf[xdf>0].unstack('type')
cum_df = xdf.assign(total=xdf.sum(1))
xdf = cum_df.reset_index().fillna(0)

In [33]:
cum_df

type                    confirmed  probable  total
Report Date DHB                                   
2020-02-26  Auckland          1.0       NaN    1.0
2020-02-28  Auckland          1.0       NaN    1.0
            Southern          1.0       NaN    1.0
2020-03-02  Auckland          1.0       NaN    1.0
            Southern          1.0       NaN    1.0
...                           ...       ...    ...
2020-04-06  Waikato         138.0      18.0  156.0
            Wairarapa         6.0       2.0    8.0
            Waitemata       109.0      50.0  159.0
            West Coast        4.0       NaN    4.0
            Whanganui         7.0       NaN    7.0

[418 rows x 3 columns]

In [ ]:
sdf = xdf.sum(0, level='Report Date')
xdf = xdf.append(sdf


In [39]:
totals_df = cum_df.sum(0, level='Report Date')
totals_df

type,confirmed,probable,total
Report Date,,,
2020-02-26,1.0,0.0,1.0
2020-02-28,2.0,0.0,2.0
2020-03-02,3.0,0.0,3.0
2020-03-04,5.0,0.0,5.0
2020-03-05,5.0,2.0,7.0
2020-03-06,6.0,2.0,8.0
2020-03-12,7.0,2.0,9.0
2020-03-13,8.0,2.0,10.0
2020-03-14,9.0,2.0,11.0


In [43]:
xdf=cum_df.fillna(0).append(totals_df.assign(DHB='total').set_index('DHB', append=True), sort=True).reset_index()
px.line(xdf, x='Report Date', y='total', color='DHB', hover_data=['confirmed', 'probable'], log_y=True)

In [63]:
xdf= cum_df.total.div(totals_df.total, level='Report Date').loc['2020-03-26':].reset_index().rename({'total':'percentage'})

In [64]:
fig=px.bar(xdf, x='Report Date', y='total', color='DHB', category_orders={'DHB':DHBS})
fig

In [17]:
df.groupby('DHB').count()

NameError: name 'df' is not defined

In [ ]:
cdf.groupby(['Report Date', 'DHB']).agg(count=('DHB', 'count')).cumsum()

In [ ]:
px.bar(cdf.groupby(['Report Date', 'DHB']).agg(count=('DHB', 'count')).reset_index(), x='Report Date', y='count', color='DHB')

In [ ]:
df2.groupby('Report Date').agg(count=('DHB', 'count'))

In [ ]:
df2.groupby('DHB').count()